In [182]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import random
import time

In [183]:
# I was using a file full of Michelin restaurants Yelp urls here

with open('/Users/lawrencewang/Downloads/restaurant_urls.csv') as f:
  urls = list(csv.reader(f))

In [184]:
urls[0][0]

'https://www.yelp.com/biz/15-east-restaurant-new-york'

In [185]:
def url_attributes(url):
    key_list = []
    value_list = []
    nest_dict = {}
    
    soup = BeautifulSoup(requests.get(url).text, "lxml")
    attributes = soup.find_all('div', {'class': 'short-def-list'})[0]
    keys = attributes.find_all('dt')
    values = attributes.find_all('dd')

    
    for key in keys:
        key_list.append(key.string.strip().replace(" ",""))
    key_list
    
    
    for value in values:
        value_list.append(value.string.strip().replace(" ",""))
    value_list
    
    
    for i in range(len(key_list)):
        nest_dict[key_list[i]] = value_list[i]

    return nest_dict


In [186]:
url_attributes(urls[1][0])

{'TakesReservations': 'Yes',
 'Delivery': 'No',
 'Take-out': 'No',
 'AcceptsCreditCards': 'Yes',
 'Parking': 'Street',
 'BikeParking': 'Yes',
 'GoodforKids': 'No',
 'GoodforGroups': 'Yes',
 'Attire': 'Dressy',
 'Ambience': 'Classy',
 'NoiseLevel': 'Average',
 'Alcohol': 'FullBar',
 'OutdoorSeating': 'Yes',
 'Wi-Fi': 'No',
 'HasTV': 'No',
 'Caters': 'No'}

### I split my urls into blocks in case the scraping ran into an error

In [188]:
urls_1 = urls[0:26]
urls_2 = urls[26:51]
urls_3 = urls[51:76]
urls_4 = urls[76:101]
urls_5 = urls[101:126]
urls_6 = urls[126:151]
urls_7 = urls[151:]

In [198]:
def populate(url_list):
    
    counter = 0
    for item in url_list:
        df = pd.DataFrame(columns = ['name', 'attributes', 'review_count',
                             'stars', 'RestaurantsPriceRange2'])
        
        soup = BeautifulSoup(requests.get(item[0]).text, "lxml")
    
        df = df.append({'name': 'Unknown' if soup.select('h1.biz-page-title.embossed-text-white')[0].string is None else\
                        soup.select('h1.biz-page-title.embossed-text-white')[0].string.encode('ascii','ignore').strip(),
                        
                        'attributes': url_attributes_alt(soup), 
                        
                        'review_count': 'Unknown' if soup.find_all('span', {'itemprop': 'reviewCount'})[0].string is None else\
                        soup.find_all('span', {'itemprop': 'reviewCount'})[0].string,
                        
                        'stars': 'Unknown' if soup.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content') is None else\
                        soup.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content'),
                        
                        'RestaurantsPriceRange2': 'Unknown' if soup.find_all('span', {'class': 'business-attribute price-range'})[0].string is None else\
                        soup.find_all('span', {'class': 'business-attribute price-range'})[0].string},
                       
                        ignore_index=True)
        

        
        df.to_csv('/Users/lawrencewang/Downloads/'+str(counter)+'.csv', index=False)
        counter += 1
        
        
        print(item)
        time.sleep(4+6*random.random())
        
        
        


### There are about 5 restaurants that give errors, so I adjusted the indexes of the url lists to skip those problem urls. Retrospectively, using a try/except in the function would have worked better

In [ ]:
populate(urls_1)

### Concatenating all the csv files into a dataframe

In [208]:
import glob
import os
path = r'/Users/lawrencewang/chi19_ds10/curriculum/project-03/Michelins'                     
all_files = glob.glob(os.path.join(path, "*.csv"))     
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [212]:
concatenated_df['Range'] = concatenated_df.RestaurantsPriceRange2.apply(lambda x: len(x))

In [214]:
concatenated_df = concatenated_df.drop('RestaurantsPriceRange2', axis = 1)
concatenated_df.head()

,name,attributes,review_count,stars,Range
0,b'15 East',"{'TakesReservations': 'Yes', 'Delivery': 'No',...",619,4.0,4
1,b'A',"{'TakesReservations': 'Yes', 'Delivery': 'No',...",285,3.5,3
2,Unknown,"{'TakesReservations': 'Yes', 'Delivery': 'No',...",523,4.0,4
3,Unknown,"{'TakesReservations': 'Yes', 'Delivery': 'No',...",309,4.0,4
4,Unknown,"{'VeganOptions': 'Yes', 'TakesReservations': '...",340,4.0,3


### Breaking out the attributes from the dictionary

In [215]:
concatenated_df["attributes"] = concatenated_df["attributes"].apply(lambda x : dict(eval(x)))
df_breakout = concatenated_df["attributes"].apply(pd.Series)

In [216]:
df_final = pd.concat([concatenated_df, df_breakout], axis=1).drop('attributes', axis=1)

,name,review_count,stars,Range,TakesReservations,Delivery,Take-out,AcceptsCreditCards,AcceptsApplePay,GoodFor,...,AcceptsCryptocurrency,HighChairs,KetoOptions,Drive-Thru,OpentoAll,VegetarianOptions,GroupBookings,HappyHourSpecials,Music,GoodforWorking
0,b'15 East',619,4.0,4,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b'A',285,3.5,3,Yes,No,No,Yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unknown,523,4.0,4,Yes,No,No,Yes,NaN,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unknown,309,4.0,4,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Unknown,340,4.0,3,Yes,No,No,Yes,Yes,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,b'Minetta',2088,4.0,3,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,b'The',931,4.5,4,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,b'Momofuku',269,4.5,4,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,b'The Musket',410,4.5,3,Yes,No,No,Yes,NaN,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Unknown,462,4.0,2,Yes,No,No,Yes,No,Dinner,...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
df_final.head(3)

,name,review_count,stars,Range,TakesReservations,Delivery,Take-out,AcceptsCreditCards,AcceptsApplePay,GoodFor,...,AcceptsCryptocurrency,HighChairs,KetoOptions,Drive-Thru,OpentoAll,VegetarianOptions,GroupBookings,HappyHourSpecials,Music,GoodforWorking
0,b'15 East',619,4.0,4,Yes,No,No,Yes,No,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b'A',285,3.5,3,Yes,No,No,Yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unknown,523,4.0,4,Yes,No,No,Yes,NaN,Dinner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
df_final.columns

Index(['name', 'review_count', 'stars', 'Range', 'TakesReservations',
       'Delivery', 'Take-out', 'AcceptsCreditCards', 'AcceptsApplePay',
       'GoodFor', 'Parking', 'BikeParking', 'GoodforKids', 'GoodforGroups',
       'Attire', 'Ambience', 'NoiseLevel', 'Alcohol', 'OutdoorSeating',
       'Wi-Fi', 'HasTV', 'Caters', 'GenderNeutralRestrooms',
       'GoodForHappyHour', 'AcceptsGooglePay', 'WaiterService', 'VeganOptions',
       'WheelchairAccessible', 'DogsAllowed', 'GoodForDancing', 'BestNights',
       'CoatCheck', 'Smoking', 'HasPoolTable', 'AcceptsCryptocurrency',
       'HighChairs', 'KetoOptions', 'Drive-Thru', 'OpentoAll',
       'VegetarianOptions', 'GroupBookings', 'HappyHourSpecials', 'Music',
       'GoodforWorking'],
      dtype='object')

In [219]:
import pickle
with open('michelin_df', 'wb') as to_write:
    pickle.dump(df_final, to_write)